In [1]:
import math
import random
import pandas as pd
import numpy as np
import tkinter as tk


from PIL import ImageTk, Image
from scoring_frames import *
from spades import *
from spades_apr_4 import *
from spades_logic_2 import *

--------------------------------------------------------------------------------------------------------------
## Let's initialize this with
# Bids
## and
# A score_frame
--------------------------------------------------------------------------------------------------------------

In [2]:
deck_bids = initialize_spades(spades_deal(shuffled(spades_deck())))
p1_bid = deck_bids[2]
p2_bid = deck_bids[4]
p3_bid = deck_bids[6]
p4_bid = deck_bids[8]
        
score_frame

Bids
Player 1    3
Player 2    1
Player 3    2
Player 4    4
Winner    nan


 --------------------------------------------------------------------------------------------------------------
# Now let's create a function to update the frame with rounds
 --------------------------------------------------------------------------------------------------------------

In [3]:

                         
#test = add_frame(1, score_frame, 1, 2, 3, 4, 1)
#test

-------------------------------------------------------------------------------------------------------------------------------
# Let's try adding rows in a loop 
-------------------------------------------------------------------------------------------------------------------------------

In [4]:
#for num in range(1,14):
#    if num == 1:
#        loop_test = add_frame(num, score_frame, 1 * num, 1 * num, 1 * num, 1 * num, 2)
        
#    if num > 1:
#        loop_test = add_frame(num, loop_test, 1 * num, 1 * num, 1 * num, 1 * num, 3)
    

#loop_test

-------------------------------------------------------------------------------------------------------------------------------
# Now let's create a function to calculate the round score
# and add it to the loop_test dataframe
-------------------------------------------------------------------------------------------------------------------------------

In [5]:
#wins = add_calculation(loop_test)
#wins

In [18]:
class Cards:
    
    def __init__(self):
        deck = initialize_spades(spades_deal(shuffled(spades_deck())))
        
        self.spades_break = 1
        
        self.p1 = sorted(set(deck[1]))
        self.p1_bid = deck[2]
        
        
        self.p2 = sorted(set(deck[3]))
        self.p2_bid = deck[4]
                
        
        self.p3 = sorted(set(deck[5]))
        self.p3_bid = deck[6]
        
        
        self.p4 = sorted(set(deck[7]))
        self.p4_bid = deck[8]
                
        self.team1_bids = self.p1_bid, self.p3_bid
        self.team2_bids = self.p2_bid, self.p4_bid
        
        self.suits = {"Hearts", "Clubs", "Diamonds", "Spades"}
    
        self.score_frame = pd.DataFrame({"Bids":[p1_bid, p2_bid, p3_bid, p4_bid,\
    "nan"],"Wins":[0, 0, 0, 0, 0], "Bags":[0, 0, 0, 0, 0], "Score":[0,0,0,0,0]}, index = ["Player 1", "Player 2", "Player 3", "Player 4","Winner"])
        
    
    def add_frame(self, n, df, p1_disc, p2_disc, p3_disc, p4_disc, winner):
        """ This method is used to update a data frame with a new column, while accounting for the winner"""
        frame = pd.DataFrame({"Hand " + str(n):[p1_disc, p2_disc, p3_disc, p4_disc,\
        winner ]}, index = ["Player 1", "Player 2", "Player 3", "Player 4","Winner"])
        winner = frame["Hand " + str(n)].loc["Winner"]
        df["Wins"]["Player " + str(winner)] += 1

        return_frame = pd.concat([df, frame], axis = 1)
        return return_frame
    def add_calculation(self, df):
        """ This method is to be used at the end of a round to calculate the scores for both teams"""
        for num in range(1,4):
            if num == 3:
                return df
            if num == 1:
                mate = 3
            if num == 2:
                mate = 4
            
            wins = df["Wins"]["Player " + str(num)]
            bids = df["Bids"]["Player " + str(num)]
            bags = wins - bids
            
            mate_wins = df["Wins"]["Player " + str(mate)]
            mate_bids = df["Bids"]["Player " + str(mate)]
            mate_bags = mate_wins - mate_bids
            
            team_wins = wins + mate_wins
            team_bids = bids + mate_bids
            team_bags = bags + mate_bags
            
        
            if team_wins < team_bids and bids != 0 and mate_bids != 0:
                df["Score"]["Player " + str(num)] -= 50
                continue
                
            if team_wins >= team_bids and bids != 0 and mate_bids != 0:
                df["Score"]["Player " + str(num)] +=  team_bids * 10
                df["Bags"]["Player " + str(num)] += team_bags
                continue
            
            if team_wins == team_bids and bids != 0 and mate_bids != 0:
                df["Score"]["Player " + str(num)] +=  team_bids * 10
                df["Bags"]["Player " + str(num)] += 0
                continue
                
            if bids == 0 and team_wins >= team_bids:
                if wins > 0:
                    df["Score"]["Player " + str(num)] +=  (team_bids * 10) - 100
                    df["Bags"]["Player " + str(num)] += team_bags
                    
                if wins == 0:
                    df["Score"]["Player " + str(num)] +=  (team_bids * 10) + 100
                    df["Bags"]["Player " + str(num)] += team_bags
                                
            if mate_bids == 0 and team_wins >= team_bids:
                if mate_wins > 0:
                    df["Score"]["Player " + str(num)] +=  (team_bids * 10) - 100
                    df["Bags"]["Player " + str(num)] += team_bags
                    
                if mate_wins == 0:
                    df["Score"]["Player " + str(num)] +=  (team_bids * 10) + 100
                    df["Bags"]["Player " + str(num)] += team_bags
                    
            if bids == 0 and team_wins <= team_bids:
                if wins > 0:
                    df["Score"]["Player " + str(num)] -=  150
                    df["Bags"]["Player " + str(num)] += team_bags
                    
                if wins == 0:
                    df["Score"]["Player " + str(num)] +=  50
                    df["Bags"]["Player " + str(num)] += team_bags
                                
            if mate_bids == 0 and team_wins <= team_bids:
                if mate_wins > 0:
                    df["Score"]["Player " + str(num)] -=  150
                    df["Bags"]["Player " + str(num)] += team_bags
                    
                if mate_wins == 0:
                    df["Score"]["Player " + str(num)] -=  150 
                    df["Bags"]["Player " + str(num)] += team_bags
            return df
            
    def turn(self, p1, p2, p3, p4):
        """ This method determines who will open bidding/discarding at the beginning of a round"""
        for enum, face, suit in self.p1:
            if face == 2 and suit == "Clubs":
                return 1
        for enum, face, suit in self.p2:
            if face == 2 and suit == "Clubs":
                return 2
        for enum, face, suit in self.p3:
            if face == 2 and suit == "Clubs":
                return 3
        for enum, face, suit in self.p4:
            if face == 2 and suit == "Clubs":
                return 4
                
    def hand_call(self, p_hand):
        """ This method returns a sorted list of enumerators for a hand, it is to be used in conjunction with GUI methods"""
        enums = []
        for enum, face, suit in sorted(p_hand):
            enums.append(str(enum))
        return sorted(set(enums))
          
        
    def hearts(self, p_hand):
        """ This method counts hearts, and will help in logic for discards"""
        count = counter(p_hand)
        return count[0]
        
    def clubs(self, p_hand):
        """ This method counts clubs, and will help in logic for discards"""
        count = counter(p_hand)
        return count[1]
        
    def diamonds(self, p_hand):
        """ This method counts diamonds, and will help in logic for discards"""
        count = counter(p_hand)
        return count[2]
    
    def spades(self, p_hand):
        """ This method counts spades, and will help in logic for discards"""
        count = counter(p_hand)
        return count[3]
    
    def max_suit(self, hearts, clubs, diamonds, spades):
        """ This method determines the suit with the most cards in a given hand"""
        if(hearts >= clubs and hearts >= diamonds and hearts >= spades):
            return "Hearts"
        if(clubs >= hearts and clubs >= diamonds and clubs >= spades):
            return "Clubs"
        if(diamonds >= hearts and diamonds >= clubs and diamonds >= spades):
            return "Diamonds"
        if(spades >= hearts and spades >= clubs and spades >= diamonds):
            return "Spades"
        
    def min_suit(self, hearts, clubs, diamonds, spades):
        """ This method determines the suit with the least cards in a given hand"""
        if(hearts <= clubs and hearts <= diamonds and hearts <= spades):
            return "Hearts"
        if(clubs <= hearts and clubs <= diamonds and clubs <= spades):
            return "Clubs"
        if(diamonds <= hearts and diamonds <= clubs and diamonds <= spades):
            return "Diamonds"
        if(spades <= hearts and spades <= clubs and spades <= diamonds):
            return "Spades"

                
                             

In [19]:
class Discard(Cards):
    def __init__(self):
        Cards.__init__(self)
        highs = ["Jack","Queen","King","Ace"]
        mids = [7,8,9,10]
        lows = [2,3,4,5,6]
    def flog(self, p_hand, p_bid, min_suit, max_suit, spades_break):
        """ This method is to be used in determining an automated discard
        for a player taking the first turn
        
        Args:
            p_hand(list of tuples) - The hand of the player discarding
            p_bid(int) - The bid of the player discarding
            min_suit(str) - The suit with the least cards
            max_suit(str) - The suit with the most cards
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            
        Returns:
            p_hand(list of tuples) - The cards in a player's hand with the discard removed
            discard(tuple) - An enumerator, face value, and suit for the card discarded
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            hand_call(set of string integers) - The left over enumurators to be used in GUI environment
        """
        hand_call = self.hand_call(p_hand)
        while spades_break == 1:
            if p_bid == 0:
                for enum, face, suit in sorted(p_hand):
                    if suit != "Spades":
                        if (suit == min_suit and face != "Ace" 
                                         or face != "King"
                                         or face != "Queen" 
                                         or face != "Jack"):
                            discard = (enum, face, suit)
                            p_hand.remove(discard)
                            hand_call.remove(str(enum))
                            return p_hand, discard, 1, hand_call
                        
                        if (suit == max_suit and face != "Ace" 
                                         or face != "King"
                                         or face != "Queen" 
                                         or face != "Jack"):
                            discard = (enum, face, suit)
                            p_hand.remove(discard)
                            hand_call.remove(str(enum))
                            return p_hand, discard, 1, hand_call
                        
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
                
            if p_bid != 0:
                for enum, face, suit in p_hand:
                    if suit != "Spades":
                        if (suit == max_suit and face == "Ace" 
                                         or face == "King"
                                         or face == "Queen" 
                                         or face == "Jack"):
                            discard = (enum, face, suit)
                            p_hand.remove(discard)
                            hand_call.remove(str(enum))
                            return p_hand, discard, 1, hand_call
                            
                        if (suit == min_suit and face == "Ace" 
                                         or face == "King"
                                         or face == "Queen" 
                                         or face == "Jack"):
                            discard = (enum, face, suit)
                            p_hand.remove(discard)
                            hand_call.remove(str(enum))
                            return p_hand, discard, 1, hand_call
                                                    
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        return p_hand, discard, 1
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
        while spades_break == 0:
            if p_bid == 0:
                for enum, face, suit in p_hand:
                    if (suit == min_suit and face != "Ace" 
                                         or face != "King"
                                         or face != "Queen" 
                                         or face != "Jack"):
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                        
                    if (suit == max_suit and face != "Ace" 
                                         or face != "King"
                                         or face != "Queen" 
                                         or face != "Jack"):
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                        
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
                    
                    
            if p_bid != 0:
                for enum, face, suit in p_hand:
                    
                    if (suit == max_suit and face == "Ace" 
                                         or face == "King"
                                         or face == "Queen" 
                                         or face == "Jack"):
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call                     
                                    
                    if (suit == min_suit and face == "Ace" 
                                         or face == "King"
                                         or face == "Queen" 
                                         or face == "Jack"):
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                                                            
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
                    
                    
    def slog(self, p_hand, p_bid, mate_bid, first_disc, spades_break):
        """ This method is to be used in determining an automated discard
        for a player taking the second turn
        
        Args:
            p_hand(list of tuples) - The hand of the player discarding
            p_bid(int) - The bid of the player discarding
            mate_bid(int) - The bid of the teammate for the player discarding
            first_disc(tuple) - The card discarded in the first turn
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            
        Returns:
            p_hand(list of tuples) - The cards in a player's hand with the discard removed
            discard(tuple) - An enumerator, face value, and suit for the card discarded
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            hand_call(set of string integers) - The left over enumurators to be used in GUI environment
        """
        set_suit = first_disc[2]
        hand_call = self.hand_call(p_hand)
        while spades_break == 1:
            if p_bid != 0:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum > first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                        
                    if suit == set_suit and enum < first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
            if p_bid == 0:
                for enum, face, suit in p_hand:
                    if suit == set_suit and enum < first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                        
                    if suit == set_suit and enum > first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
        while spades_break == 0:
            if p_bid != 0:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum > first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                        
                    if suit == set_suit and enum < first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
             
            if p_bid == 0:
                for enum, face, suit in p_hand:
                    if suit == set_suit and enum < first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                        
                    if suit == set_suit and enum > first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
                for enum, face, suit in p_hand:
                    if enum < first_disc[0]:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    if enum > first_disc and suit != set_suit and suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    if face in lows:    
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:    
                    if face in mids:    
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:    
                    if face in highs:    
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
    def tlog(self, p_hand, p_bid, mate_bid, mate_disc, opp_disc, spades_break):
        """ This method is to be used in determining an automated discard
        for a player taking the third turn
        
        Args:
            p_hand(list of tuples) - The hand of the player discarding
            p_bid(int) - The bid of the player discarding
            mate_bid(int) - The bid of the teammate for the player discarding
            mate_disc(tuple) - The card discarded in the first turn
            opp_disc(tuble) - The card discarded in the second turn
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            
        Returns:
            p_hand(list of tuples) - The cards in a player's hand with the discard removed
            discard(tuple) - An enumerator, face value, and suit for the card discarded
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            hand_call(set of string integers) - The left over enumurators to be used in GUI environment
        """
        set_suit = mate_disc[2]
        mate_enum = mate_disc[0]
        opp_suit = opp_disc[2]
        opp_enum = opp_disc[0]
        hand_call = self.hand_call(p_hand)
        while spades_break == 1:
            if p_bid != 0 and mate_enum < opp_enum:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum > opp_enum:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                     
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
            if p_bid != 0 and mate_enum > opp_enum:
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call 
                for enum, face, suit in p_hand:
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
            if p_bid == 0:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum < opp_enum or enum < mate_enum:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
            
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
                for enum, face, suit in p_hand:    
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call            
                    
        while spades_break == 0:
            if p_bid != 0 and mate_enum < opp_enum:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum > opp_enum:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                     
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call                    
                    
            if p_bid != 0 and mate_enum > opp_enum:
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    if suit != set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
            if p_bid == 0:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum < opp_enum or enum < mate_enum:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
            
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
                for enum, face, suit in p_hand:    
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
            for enum, face, suit in p_hand:
                discard = (enum, face, suit)
                p_hand.remove(discard)
                hand_call.remove(str(enum))
                return p_hand, discard, 0, hand_call
            
    
    def four_log(self, p_hand, p_bid, mate_bid, mate_disc, opp_disc, opp2_disc, spades_break):
        """ This method is to be used in determining an automated discard
        for a player taking the third turn
        
        Args:
            p_hand(list of tuples) - The hand of the player discarding
            p_bid(int) - The bid of the player discarding
            mate_bid(int) - The bid of the teammate for the player discarding
            mate_disc(tuple) - The card discarded in the first turn
            opp_disc(tuble) - The card discarded in the second turn
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            
        Returns:
            p_hand(list of tuples) - The cards in a player's hand with the discard removed
            discard(tuple) - An enumerator, face value, and suit for the card discarded
            spades_break(int) - Either a 1 or 0, used for boolean purposes
            hand_call(set of string integers) - The left over enumurators to be used in GUI environment
        """
        mate_suit = mate_disc[2]
        mate_enum = mate_disc[0]
        
        set_suit = opp_disc[2]
        opp_enum = opp_disc[0]
        opp2_suit = opp2_disc[2]
        opp2_enum = opp2_disc[0] 
        
        hand_call = self.hand_call(p_hand)
        
        while spades_break == 1:
            if (p_bid != 0 and mate_enum < opp_enum or mate_enum < opp2_enum):
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum > opp_enum and enum > opp2_enum :
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                     
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
            if (p_bid != 0 
                and mate_enum > opp_enum 
                and mate_enum > opp_enum
                and mate_suit == "Spades"
                or mate_suit == set_suit):
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call 
                    
                for enum, face, suit in p_hand:
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
            if p_bid == 0:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if (suit == set_suit 
                        and (enum < opp_enum 
                        or enum < mate_enum 
                        or enum < opp2_enum)):
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
            
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    
                for enum, face, suit in p_hand:    
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 1, hand_call
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call            
                    
        while spades_break == 0:
            if p_bid != 0 and mate_enum < opp_enum or mate_enum < opp2_enum:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum > opp_enum:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                     
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                
                for enum, face, suit in p_hand:
                    if suit == "Spades":    
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
                    
            if p_bid != 0 and mate_enum > opp_enum and mate_enum > opp2_enum:
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    if suit == "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                
                for enum, face, suit in p_hand:
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
                    
            if p_bid == 0:
                for enum, face, suit in sorted(p_hand, reverse = True):
                    if suit == set_suit and enum < opp_enum or enum < mate_enum or enum < opp2_enum:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
            
                for enum, face, suit in p_hand:
                    if suit == set_suit:
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                    
                for enum, face, suit in p_hand:    
                    if suit != "Spades":
                        discard = (enum, face, suit)
                        p_hand.remove(discard)
                        hand_call.remove(str(enum))
                        return p_hand, discard, 0, hand_call
                for enum, face, suit in p_hand:
                    discard = (enum, face, suit)
                    p_hand.remove(discard)
                    hand_call.remove(str(enum))
                    return p_hand, discard, 0, hand_call
                    
            for enum, face, suit in p_hand:
                discard = (enum, face, suit)
                p_hand.remove(discard)
                hand_call.remove(str(enum))
                return p_hand, discard, 0, hand_call
            
                
    def winner(self, p1_disc, p2_disc, p3_disc, p4_disc, set_suit):
        """ This method determines the winner based on 4 discards
        Args:
            p1_disc(tuple) - Discard from first turn
            p2_disc(tuple) - Discard from second turn
            p3_disc(tuple) - Discard from third turn
            p4_disc(tuple) - Discard from fourth turn
            
        Returns:
            winner(int) - An integer 1-4 representing the player that won"""
        for a, b, c in sorted(set([p1_disc, p2_disc, p3_disc, p4_disc]), reverse = True):
            if c == set_suit or c == "Spades":
                if (a,b,c) == p1_disc:
                    winner = 1
                if(a,b,c) == p2_disc:
                    winner = 2
                if(a,b,c) == p3_disc:
                    winner = 3
                if(a,b,c) == p4_disc:
                    winner = 4
                return winner
            

In [20]:
for i in range(1,5):    
    teste = Cards()
    p1 = teste.p1
    p2 = teste.p2
    p3 = teste.p3
    p4 = teste.p4
    p1_bid = teste.p4_bid
    
    turn = teste.turn(p1, p2, p3, p4)
    
    hearts = teste.hearts(p1)
    clubs = teste.clubs(p1)
    diamonds = teste.diamonds(p1)
    spades = teste.spades(p1)
    
    max_suit = teste.max_suit(hearts, clubs, diamonds, spades)
    min_suit = teste.min_suit(hearts, clubs, diamonds, spades)
    
    total = hearts + clubs + diamonds + spades
    
    hand_call = teste.hand_call(p1)
    
    
    print("Turn=", turn, "Total=", total,  "Counts =", hearts, clubs, diamonds, spades,"\n")
    print("Hand call =", hand_call)
    print("Max suit =", max_suit, "Min suit =", min_suit)

Turn= 1 Total= 13 Counts = 5 3 3 2 

Hand call = ['10', '14', '15', '26', '27', '35', '38', '46', '52', '6', '7', '8', '9']
Max suit = Hearts Min suit = Spades
Turn= 3 Total= 13 Counts = 1 5 4 3 

Hand call = ['1', '15', '17', '18', '19', '23', '32', '33', '35', '38', '40', '44', '51']
Max suit = Clubs Min suit = Hearts
Turn= 1 Total= 13 Counts = 1 5 4 3 

Hand call = ['14', '18', '2', '23', '24', '26', '30', '32', '34', '39', '40', '49', '51']
Max suit = Clubs Min suit = Hearts
Turn= 1 Total= 13 Counts = 6 2 3 2 

Hand call = ['1', '11', '12', '14', '15', '27', '32', '33', '43', '5', '50', '7', '8']
Max suit = Hearts Min suit = Clubs


In [22]:
for i in range(1,14):    
    log_test = Cards()
    disc_test = Discard()
    
    bid1 = log_test.p1_bid
    bid2 = log_test.p2_bid
    bid3 = log_test.p3_bid
    bid4 = log_test.p4_bid    
    
    if i == 1:
        frame = log_test.score_frame
        spades_break = 1
        hand1 = log_test.p1
        hand2 = log_test.p2
        hand3 = log_test.p3
        hand4 = log_test.p4
        
        hand_call1 = log_test.hand_call(hand1)
        hand_call2 = log_test.hand_call(hand2)
        hand_call3 = log_test.hand_call(hand3)
        hand_call4 = log_test.hand_call(hand4)
        
        hearts = log_test.hearts(hand1)
        clubs = log_test.clubs(hand1)
        diamonds = log_test.diamonds(hand1)
        spades = log_test.spades(hand1)
     
        hearts2 = log_test.hearts(hand2)
        clubs2 = log_test.clubs(hand2)
        diamonds2 = log_test.diamonds(hand2)
        spades2 = log_test.spades(hand2)
        
        hearts3 = log_test.hearts(hand3)
        clubs3 = log_test.clubs(hand3)
        diamonds3 = log_test.diamonds(hand3)
        spades3 = log_test.spades(hand3)
        
        hearts4 = log_test.hearts(hand4)
        clubs4 = log_test.clubs(hand4)
        diamonds4 = log_test.diamonds(hand4)
        spades4 = log_test.spades(hand4)
        
        min_suit_1 = log_test.min_suit(hearts, clubs, diamonds, spades)
        max_suit_1 = log_test.max_suit(hearts, clubs, diamonds, spades)
    
        min_suit_2 = log_test.min_suit(hearts2, clubs2, diamonds2, spades2)
        max_suit_2 = log_test.max_suit(hearts2, clubs2, diamonds2, spades2)
    
        min_suit_3 = log_test.min_suit(hearts3, clubs3, diamonds3, spades3)
        max_suit_3 = log_test.max_suit(hearts3, clubs3, diamonds3, spades3)
        
        min_suit_4 = log_test.min_suit(hearts4, clubs4, diamonds4, spades4)
        max_suit_4 = log_test.max_suit(hearts4, clubs4, diamonds4, spades4)
    
        disc_bund = disc_test.flog(hand1, bid1, min_suit, max_suit, spades_break)
        hand1 = sorted(set(disc_bund[0]))
        disc1 = disc_bund[1]
        spades_break = disc_bund[2]  
        hand_call1 = log_test.hand_call(hand1)
        print(i,spades_break, disc1, "\n\n Cards still in hand:\n", hand1,"\n")
    
        disc_bund_2 = disc_test.slog(hand2, bid2, bid4, disc1, spades_break)
        hand2 = sorted(set(disc_bund_2[0]))
        disc2 = disc_bund_2[1]
        spades_break = disc_bund_2[2]
        hand_call2 = log_test.hand_call(hand2)    
        print(i, spades_break, disc2, "\n\n Cards still in hand:\n", hand2,"\n")
    
        disc_bund_3 = disc_test.tlog(hand3, bid3, bid1, disc1, disc2, spades_break)
        hand3 = sorted(set(disc_bund_3[0]))
        disc3 = disc_bund_3[1]
        spades_break = disc_bund[2]
        hand_call3 = log_test.hand_call(hand3)
        print(i, spades_break, disc3, "\n\n Cards still in hand:\n", hand3, "\n")
    
        disc_bund_4 = disc_test.four_log(hand4, bid4, bid2, disc2, disc1, disc3, spades_break)
        hand4 = sorted(set(disc_bund_4[0]))
        disc4 = disc_bund_4[1]
        spades_break = disc_bund_4[2]
        hand_call4 = log_test.hand_call(hand4)
        print(i, spades_break, disc4, "\n\n Cards still in hand:\n", hand4, "\n")
        
        set_suit = disc1[2]
        winner = disc_test.winner(disc1, disc2, disc3, disc4, set_suit)
        round_frame = log_test.add_frame(i, frame, disc1, disc2, disc3, disc4, winner)
        continue
        
    if i > 1:
        hearts = log_test.hearts(hand1)
        clubs = log_test.clubs(hand1)
        diamonds = log_test.diamonds(hand1)
        spades = log_test.spades(hand1)
        
        hearts2 = log_test.hearts(hand2)
        clubs2 = log_test.clubs(hand2)
        diamonds2 = log_test.diamonds(hand2)
        spades2 = log_test.spades(hand2)
        
        hearts3 = log_test.hearts(hand3)
        clubs3 = log_test.clubs(hand3)
        diamonds3 = log_test.diamonds(hand3)
        spades3 = log_test.spades(hand3)
        
        hearts4 = log_test.hearts(hand4)
        clubs4 = log_test.clubs(hand4)
        diamonds4 = log_test.diamonds(hand4)
        spades4 = log_test.spades(hand4)
        
        min_suit_1 = log_test.min_suit(hearts, clubs, diamonds, spades)
        max_suit_1 = log_test.max_suit(hearts, clubs, diamonds, spades)
    
        min_suit_2 = log_test.min_suit(hearts2, clubs2, diamonds2, spades2)
        max_suit_2 = log_test.max_suit(hearts2, clubs2, diamonds2, spades2)
    
        min_suit_3 = log_test.min_suit(hearts3, clubs3, diamonds3, spades3)
        max_suit_3 = log_test.max_suit(hearts3, clubs3, diamonds3, spades3)
        
        min_suit_4 = log_test.min_suit(hearts4, clubs4, diamonds4, spades4)
        max_suit_4 = log_test.max_suit(hearts4, clubs4, diamonds4, spades4)
     
        if winner == 1:
            disc_bund = disc_test.flog(hand1, bid1, min_suit, max_suit, spades_break)
            hand1 = sorted(set(disc_bund[0]))
            disc1 = disc_bund[1]
            spades_break = disc_bund[2]  
            hand_call1 = log_test.hand_call(hand1)
            print(i,spades_break, disc1, "\n\n Cards still in hand:\n", hand1,"\n")
    
            disc_bund_2 = disc_test.slog(hand2, bid2, bid4, disc1, spades_break)
            hand2 = sorted(set(disc_bund_2[0]))
            disc2 = disc_bund_2[1]
            spades_break = disc_bund_2[2]
            hand_call2 = log_test.hand_call(hand2)    
            print(i, spades_break, disc2, "\n\n", "Cards still in hand:\n", hand2,"\n")
    
            disc_bund_3 = disc_test.tlog(hand3, bid3, bid1, disc1, disc2, spades_break)
            hand3 = sorted(set(disc_bund_3[0]))
            disc3 = disc_bund_3[1]
            spades_break = disc_bund_3[2]
            hand_call3 = log_test.hand_call(hand3)
            print(i, spades_break, disc3, "\n\n Cards still in hand:\n", hand3, "\n")
    
            disc_bund_4 = disc_test.four_log(hand4, bid4, bid2, disc2, disc1, disc3, spades_break)
            hand4 = sorted(set(disc_bund_4[0]))
            disc4 = disc_bund_4[1]
            spades_break = disc_bund_4[2]
            hand_call4 = log_test.hand_call(hand4)
            print(i, spades_break, disc4, "\n\n Cards still in hand:\n", hand4, "\n\n\n\n\n")
            
            set_suit = disc1[2]
            winner = disc_test.winner(disc1, disc2, disc3, disc4, set_suit)
            round_frame = log_test.add_frame(i, round_frame, disc1, disc2, disc3, disc4, winner)
            continue

        if winner == 2:
            disc_bund = disc_test.flog(hand2, bid2, min_suit_2, max_suit_2, spades_break)
            hand2 = sorted(set(disc_bund[0]))
            disc2 = disc_bund[1]
            spades_break = disc_bund[2]  
            hand_call2 = log_test.hand_call(hand2)
            print(i,spades_break, disc2, "\n\n Cards still in hand:\n", hand2,"\n")
    
            disc_bund_2 = disc_test.slog(hand3, bid3, bid1, disc2, spades_break)
            hand3 = sorted(set(disc_bund_2[0]))
            disc3 = disc_bund_2[1]
            spades_break = disc_bund_2[2]
            hand_call3 = log_test.hand_call(hand3)    
            print(i, spades_break, disc3, "\n\n Cards still in hand:\n", hand3,"\n")
    
            disc_bund_3 = disc_test.tlog(hand4, bid4, bid2, disc2, disc3, spades_break)
            hand4 = sorted(set(disc_bund_3[0]))
            disc4 = disc_bund_3[1]
            spades_break = disc_bund_3[2]
            hand_call4 = log_test.hand_call(hand4)
            print(i, spades_break, disc4, "\n\n Cards still in hand:\n", hand4, "\n")
    
            disc_bund_4 = disc_test.four_log(hand1, bid1, bid3, disc3, disc1, disc2, spades_break)
            hand1 = sorted(set(disc_bund_4[0]))
            disc1 = disc_bund_4[1]
            spades_break = disc_bund_4[2]
            hand_call1 = log_test.hand_call(hand1)
            print(i, spades_break, disc1, "\n\n Cards still in hand:\n", hand1, "\n\n\n\n\n\n")
        
            set_suit = disc2[2]
            winner = disc_test.winner(disc1, disc2, disc3, disc4, set_suit)
            round_frame = log_test.add_frame(i, round_frame, disc1, disc2, disc3, disc4, winner)
            continue
            
        if winner == 3:
            disc_bund = disc_test.flog(hand3, bid3, min_suit_3, max_suit_3, spades_break)
            hand3 = sorted(set(disc_bund[0]))
            disc3 = disc_bund[1]
            spades_break = disc_bund[2]  
            hand_call3 = log_test.hand_call(hand3)
            print(i,spades_break, disc3, "\n\n Cards still in hand:\n", hand3,"\n")
    
            disc_bund_2 = disc_test.slog(hand4, bid4, bid2, disc3, spades_break)
            hand4 = sorted(set(disc_bund_2[0]))
            disc4 = disc_bund_2[1]
            spades_break = disc_bund_2[2]
            hand_call4 = log_test.hand_call(hand4)    
            print(i, spades_break, disc4, "\n\n Cards still in hand:\n", hand4,"\n")
    
            disc_bund_3 = disc_test.tlog(hand1, bid1, bid3, disc3, disc4, spades_break)
            hand1 = sorted(set(disc_bund_3[0]))
            disc1 = disc_bund_3[1]
            spades_break = disc_bund_3[2]
            hand_call1 = log_test.hand_call(hand1)
            print(i, spades_break, disc1, "\n\n Cards still in hand:\n", hand1, "\n")
    
            disc_bund_4 = disc_test.four_log(hand2, bid2, bid4, disc2, disc1, disc3, spades_break)
            hand2 = sorted(set(disc_bund_4[0]))
            disc2 = disc_bund_4[1]
            spades_break = disc_bund_4[2]
            hand_call2 = log_test.hand_call(hand2)
            print(i, spades_break, disc2, "\n\n Cards still in hand:\n", hand2, "\n\n\n\n\n")
            
            set_suit = disc3[2]
            winner = disc_test.winner(disc1, disc2, disc3, disc4, set_suit)
            round_frame = log_test.add_frame(i, round_frame, disc1, disc2, disc3, disc4, winner)
            continue
            
        if winner == 4:
            disc_bund = disc_test.flog(hand4, bid4, min_suit_4, max_suit_4, spades_break)
            hand4 = sorted(set(disc_bund[0]))
            disc4 = disc_bund[1]
            spades_break = disc_bund[2]  
            hand_call4 = log_test.hand_call(hand4)
            print(i,spades_break, disc4, "\n\n Cards still in hand:\n", hand4,"\n")
    
            disc_bund_2 = disc_test.slog(hand1, bid1, bid3, disc4, spades_break)
            hand1 = sorted(set(disc_bund_2[0]))
            disc1 = disc_bund_2[1]
            spades_break = disc_bund_2[2]
            hand_call1 = log_test.hand_call(hand1)    
            print(i, spades_break, disc1, "\n\n Cards still in hand:\n", hand1,"\n")
    
            disc_bund_3 = disc_test.tlog(hand2, bid2, bid4, disc4, disc1, spades_break)
            hand2 = sorted(set(disc_bund_3[0]))
            disc2 = disc_bund_3[1]
            spades_break = disc_bund_3[2]
            hand_call2 = log_test.hand_call(hand2)
            print(i, spades_break, disc2, "\n\n Cards still in hand:\n", hand2, "\n")
    
            disc_bund_4 = disc_test.four_log(hand3, bid3, bid1, disc1, disc2, disc3, spades_break)      
            hand3 = sorted(set(disc_bund_4[0]))
            disc3 = disc_bund_4[1]
            spades_break = disc_bund_4[2]
            hand_call3 = log_test.hand_call(hand3)
            print(i, spades_break, disc3, "\n\n Cards still in hand:\n", hand3, "\n\n\n\n")
            
            set_suit = disc4[2]
            winner = disc_test.winner(disc1, disc2, disc3, disc4, set_suit)
            round_frame = log_test.add_frame(i, round_frame, disc1, disc2, disc3, disc4, winner)
            continue
        
            

        
    

1 1 (6, 7, 'Hearts') 

 Cards still in hand:
 [(8, 9, 'Hearts'), (11, 'Queen', 'Hearts'), (14, 2, 'Clubs'), (16, 4, 'Clubs'), (17, 5, 'Clubs'), (19, 7, 'Clubs'), (20, 8, 'Clubs'), (33, 8, 'Diamonds'), (35, 10, 'Diamonds'), (37, 'Queen', 'Diamonds'), (43, 5, 'Spades'), (44, 6, 'Spades')] 

1 1 (10, 'Jack', 'Hearts') 

 Cards still in hand:
 [(1, 2, 'Hearts'), (3, 4, 'Hearts'), (5, 6, 'Hearts'), (7, 8, 'Hearts'), (29, 4, 'Diamonds'), (30, 5, 'Diamonds'), (31, 6, 'Diamonds'), (32, 7, 'Diamonds'), (47, 9, 'Spades'), (49, 'Jack', 'Spades'), (51, 'King', 'Spades'), (52, 'Ace', 'Spades')] 

1 1 (12, 'King', 'Hearts') 

 Cards still in hand:
 [(4, 5, 'Hearts'), (9, 10, 'Hearts'), (15, 3, 'Clubs'), (18, 6, 'Clubs'), (21, 9, 'Clubs'), (23, 'Jack', 'Clubs'), (24, 'Queen', 'Clubs'), (26, 'Ace', 'Clubs'), (34, 9, 'Diamonds'), (42, 4, 'Spades'), (45, 7, 'Spades'), (50, 'Queen', 'Spades')] 

1 1 (13, 'Ace', 'Hearts') 

 Cards still in hand:
 [(2, 3, 'Hearts'), (22, 10, 'Clubs'), (25, 'King', 'Clubs')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


8 0 (28, 3, 'Diamonds') 

 Cards still in hand:
 [(36, 'Jack', 'Diamonds'), (38, 'King', 'Diamonds'), (41, 3, 'Spades'), (46, 8, 'Spades'), (48, 10, 'Spades')] 

8 0 (33, 8, 'Diamonds') 

 Cards still in hand:
 [(19, 7, 'Clubs'), (35, 10, 'Diamonds'), (37, 'Queen', 'Diamonds'), (43, 5, 'Spades'), (44, 6, 'Spades')] 

8 0 (30, 5, 'Diamonds') 

 Cards still in hand:
 [(31, 6, 'Diamonds'), (32, 7, 'Diamonds'), (49, 'Jack', 'Spades'), (51, 'King', 'Spades'), (52, 'Ace', 'Spades')] 

8 0 (45, 7, 'Spades') 

 Cards still in hand:
 [(21, 9, 'Clubs'), (23, 'Jack', 'Clubs'), (24, 'Queen', 'Clubs'), (26, 'Ace', 'Clubs'), (50, 'Queen', 'Spades')] 




9 0 (21, 9, 'Clubs') 

 Cards still in hand:
 [(23, 'Jack', 'Clubs'), (24, 'Queen', 'Clubs'), (26, 'Ace', 'Clubs'), (50, 'Queen', 'Spades')] 

9 0 (36, 'Jack', 'Diamonds') 

 Cards still in hand:
 [(38, 'King', 'Diamonds'), (41, 3, 'Spades'), (46, 8, 'Spades'), (48, 10, 'Spades')] 

9 0 (19, 7, 'Clubs') 

 Cards still in hand:
 [(35, 10, 'Diamonds')

In [23]:
round_frame

Bids  Wins  Bags  Score              Hand 1               Hand 2  \
Player 1    5     2     0      0      (6, 7, Hearts)  (11, Queen, Hearts)   
Player 2    1     5     0      0  (10, Jack, Hearts)       (1, 2, Hearts)   
Player 3    3     2     0      0  (12, King, Hearts)       (4, 5, Hearts)   
Player 4    3     4     0      0   (13, Ace, Hearts)       (2, 3, Hearts)   
Winner    nan     0     0      0                   4                    1   

                   Hand 3           Hand 4             Hand 5  \
Player 1   (8, 9, Hearts)   (20, 8, Clubs)     (14, 2, Clubs)   
Player 2   (7, 8, Hearts)   (3, 4, Hearts)    (47, 9, Spades)   
Player 3  (9, 10, Hearts)  (42, 4, Spades)     (15, 3, Clubs)   
Player 4  (40, 2, Spades)  (22, 10, Clubs)  (25, King, Clubs)   
Winner                  4                3                  2   

                     Hand 6               Hand 7             Hand 8  \
Player 1     (16, 4, Clubs)       (17, 5, Clubs)  (33, 8, Diamonds)   
Player 2     (5, 6, Hearts)    (29, 4, Diamonds)  (30, 5, Diamonds)   
Player 3     (18, 6, Clubs)    (34, 9, Diamonds)    (45, 7, Spades)   
Player 4  (27, 2, Diamonds)  (39, Ace, Diamonds)  (28, 3, Diamonds)   
Winner                    2                    4                  3   

                        Hand 9               Hand 10             Hand 11  \
Player 1        (19, 7, Clubs)       (43, 5, Spades)  (35, 10, Diamonds)   
Player 2    (49, Jack, Spades)     (31, 6, Diamonds)   (32, 7, Diamonds)   
Player 3        (21, 9, Clubs)     (23, Jack, Clubs)  (24, Queen, Clubs)   
Player 4  (36, Jack, Diamonds)  (38, King, Diamonds)     (41, 3, Spades)   
Winner                       2                     1                   4   

                      Hand 12                Hand 13  
Player 1      (44, 6, Spades)  (37, Queen, Diamonds)  
Player 2   (51, King, Spades)      (52, Ace, Spades)  
Player 3  (50, Queen, Spades)       (26, Ace, Clubs)  
Player 4      (46, 8, Spades)       (48, 10, Spades)  
Winner                      2                      2

In [24]:
score_frame = log_test.add_calculation(round_frame)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
score_frame


Bids  Wins  Bags  Score              Hand 1               Hand 2  \
Player 1    5     2     0    -50      (6, 7, Hearts)  (11, Queen, Hearts)   
Player 2    1     5     5     40  (10, Jack, Hearts)       (1, 2, Hearts)   
Player 3    3     2     0      0  (12, King, Hearts)       (4, 5, Hearts)   
Player 4    3     4     0      0   (13, Ace, Hearts)       (2, 3, Hearts)   
Winner    nan     0     0      0                   4                    1   

                   Hand 3           Hand 4             Hand 5  \
Player 1   (8, 9, Hearts)   (20, 8, Clubs)     (14, 2, Clubs)   
Player 2   (7, 8, Hearts)   (3, 4, Hearts)    (47, 9, Spades)   
Player 3  (9, 10, Hearts)  (42, 4, Spades)     (15, 3, Clubs)   
Player 4  (40, 2, Spades)  (22, 10, Clubs)  (25, King, Clubs)   
Winner                  4                3                  2   

                     Hand 6               Hand 7             Hand 8  \
Player 1     (16, 4, Clubs)       (17, 5, Clubs)  (33, 8, Diamonds)   
Player 2     (5, 6, Hearts)    (29, 4, Diamonds)  (30, 5, Diamonds)   
Player 3     (18, 6, Clubs)    (34, 9, Diamonds)    (45, 7, Spades)   
Player 4  (27, 2, Diamonds)  (39, Ace, Diamonds)  (28, 3, Diamonds)   
Winner                    2                    4                  3   

                        Hand 9               Hand 10             Hand 11  \
Player 1        (19, 7, Clubs)       (43, 5, Spades)  (35, 10, Diamonds)   
Player 2    (49, Jack, Spades)     (31, 6, Diamonds)   (32, 7, Diamonds)   
Player 3        (21, 9, Clubs)     (23, Jack, Clubs)  (24, Queen, Clubs)   
Player 4  (36, Jack, Diamonds)  (38, King, Diamonds)     (41, 3, Spades)   
Winner                       2                     1                   4   

                      Hand 12                Hand 13  
Player 1      (44, 6, Spades)  (37, Queen, Diamonds)  
Player 2   (51, King, Spades)      (52, Ace, Spades)  
Player 3  (50, Queen, Spades)       (26, Ace, Clubs)  
Player 4      (46, 8, Spades)       (48, 10, Spades)  
Winner                      2                      2